# Import Packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Importing Packages 
import numpy as np
import pandas as pd

#Importing packages for cross_validation 
from sklearn.model_selection import cross_val_score

#for modeling 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from collections import Counter

#for feature selection
from sklearn.feature_selection import SelectKBest, f_classif

## Grid search cross validation
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold


# Read Test and Train Data

In [4]:
train = pd.read_csv('./santander-customer-transaction-prediction/train.csv')
test = pd.read_csv('./santander-customer-transaction-prediction/test.csv')

# Feature Inspection

*  The goal of this step is to check if the features are numerical/categorical and also if they have any missing values. Findings from the inspection helps to determine how the data should be processed before applying modeling.

*  Findings from the code below show that 'train' and test dataset has 201 input features and 200,000 entries, and no nulls values in the dataset. This means that  the solution approach does not require any of the pre- processing steps that are usually followed for handling missing values in a dataset.


* **Inpsect Number of Entries,Features and Data Type**

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [6]:
train.tail(5)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
199995,train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,6.1415,13.2305,3.9901,0.9388,18.0249,-1.7939,2.1661,8.5326,16.6660,-17.8661
199996,train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,4.9611,4.6549,0.6998,1.8341,22.2717,1.7337,-2.1651,6.7419,15.9054,0.3388
199997,train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,4.0651,5.4414,3.1032,4.8793,23.5311,-1.5736,1.2832,8.7155,13.8329,4.1995
199998,train_199998,0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,2.6840,8.6587,2.7337,11.1178,20.4158,-0.0786,6.7980,10.0342,15.5289,-13.9001
199999,train_199999,0,10.8762,-5.7105,12.1183,8.0328,11.5577,0.3488,5.2839,15.2058,...,8.9842,1.6893,0.1276,0.3766,15.2101,-2.4907,-2.2342,8.1857,12.1284,0.1385


In [7]:
test.describe()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,10.658737,-1.624244,10.707452,6.788214,11.076399,-5.050558,5.415164,16.529143,0.277135,7.569407,...,3.189766,7.458269,1.925944,3.322016,17.996967,-0.133657,2.290899,8.912428,15.869184,-3.246342
std,3.036716,4.040509,2.633888,2.052724,1.616456,7.869293,0.864686,3.424482,3.333375,1.231865,...,4.551239,3.025189,1.479966,3.995599,3.140652,1.429678,5.446346,0.920904,3.008717,10.398589
min,0.188700,-15.043400,2.355200,-0.022400,5.484400,-27.767000,2.216400,5.713700,-9.956000,4.243300,...,-14.093300,-2.407000,-3.340900,-11.413100,9.382800,-4.911900,-13.944200,6.169600,6.584000,-39.457800
25%,8.442975,-4.700125,8.735600,5.230500,9.891075,-11.201400,4.772600,13.933900,-2.303900,6.623800,...,-0.095000,5.166500,0.882975,0.587600,15.634775,-1.160700,-1.948600,8.260075,13.847275,-11.124000
50%,10.513800,-1.590500,10.560700,6.822350,11.099750,-4.834100,5.391600,16.422700,0.372000,7.632000,...,3.162400,7.379000,1.892600,3.428500,17.977600,-0.162000,2.403600,8.892800,15.943400,-2.725950
75%,12.739600,1.343400,12.495025,8.327600,12.253400,0.942575,6.005800,19.094550,2.930025,8.584825,...,6.336475,9.531100,2.956000,6.174200,20.391725,0.837900,6.519800,9.595900,18.045200,4.935400
max,22.323400,9.385100,18.714100,13.142000,16.037100,17.253700,8.302500,28.292800,9.665500,11.003600,...,20.359000,16.716500,8.005000,17.632600,27.947800,4.545400,15.920700,12.275800,26.538400,27.907400


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 201 entries, ID_code to var_199
dtypes: float64(200), object(1)
memory usage: 306.7+ MB


* **Number of Missing Values in the Datase**t 

In [11]:
train.isnull().sum()

ID_code    0
target     0
var_0      0
var_1      0
var_2      0
          ..
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 202, dtype: int64

In [12]:
test.isnull().sum().sum()

0

# 4. Feature Selection

* [Feature selection algorithms](https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/) help to determine which features significantly influence the output variable and then eliminate the remaining  features from consideration. This step was useful as this dataset has large number of features, being able to eliminate features that have little to no impact on the analysis reduced the complexity of the model and enabled machine learning algortihms to train faster. 

* The scikit module provides [several ways](https://scikit-learn.org/stable/modules/feature_selection.html) for identifying significant features either by evaluating the statistical correlation with the outcome variable (e.g. univariate feature selection method) or by measuring  the usefulness of a subset of feature by actually training a machine learning model on it (e.g. recursive feature elimination). 

* Considering the size of this particular dataset and the computational time usually required for recursive feature elimination, the universal feature seleciton method was adopted to select the top 25 features that are critical for modeling. Later, submissions were also made by selecting top 50, 100 and 150 features to check the effect of feature selection on accuracy and time required for computation.

* **Define Target Variable and Test Variables for Modeling**

In [13]:
train_X= train.drop(columns=['target','ID_code'],axis=1)
train_Y=train['target']
test_X= test.drop(columns=['ID_code'],axis=1)

* **Select Features to be Considered for Modeling**

In [15]:
selector = SelectKBest(f_classif, k=25)
X_new=selector.fit_transform(train_X, train_Y)
mask = selector.get_support(indices=True)
colname = train_X.columns[mask]# Names of the selected columns
colname

Index(['var_0', 'var_1', 'var_2', 'var_6', 'var_12', 'var_13', 'var_21',
       'var_22', 'var_26', 'var_34', 'var_53', 'var_76', 'var_80', 'var_81',
       'var_99', 'var_110', 'var_133', 'var_139', 'var_146', 'var_148',
       'var_165', 'var_166', 'var_174', 'var_190', 'var_198'],
      dtype='object')

* **Redifining Train and  Test Data Using the Selected Features** 

In [16]:
train_X_new = train_X[colname]
test_X_new = test_X[colname]

# 5. Train Different Classification Models using Selected Features

* All models (KNN, Logistic Regresson, Decision Tree,XGBoost and Random Forest) were trained using the train dataset with selected features. 
* To estimate the accuracy of these models when applied to unseen data, cross validation was performed for each model by splitting the train dataset (80%/20% for train/test split), fitting the model and computing the score 5 consecutive times (with different splits each time) based on ROC_AUC score. The mean of the accuracies for the 5 trials were then reported as the CV score for each model.

* **Defining models**

In [17]:
#Random Forest, Decision Tree, Logistic Regression and K Nearest Neighbours, and XGBoost models
RFmodel=RandomForestClassifier(random_state=1)
DTmodel=DecisionTreeClassifier(random_state=1)
logreg = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors = 3)
xgbm = xgb.XGBClassifier(silent=True, scale_pos_weight=1,learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)

* **Training the Models**

In [18]:
RFmodel.fit(train_X_new,train_Y)
DTmodel.fit(train_X_new,train_Y)
logreg.fit(train_X_new,train_Y)
knn.fit(train_X_new,train_Y)
xgbm.fit(train_X_new,train_Y)

/Users/prguser/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/prguser/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=10,
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.8, verbosity=1)

* **Calculating Cross Validation Scores**

In [19]:
#define a function that can be called to calculate cross validation score for each model 
def cross_val(X,x,y):
    scores = cross_val_score(X, x, y, cv=5, scoring = "roc_auc")
    return scores.mean()

In [26]:
#Calculate Cross Validation Score based on ROC_AUC by calling th ecross_val function defined before 
#RF_score = round(cross_val(RFmodel,train_X_new, train_Y) * 100, 2)
#DT_score =round(cross_val(DTmodel,train_X_new, train_Y) * 100, 2)
#logreg_score = round(cross_val(logreg,train_X_new, train_Y) * 100, 2)
#knn_score = round(cross_val(knn,train_X_new, train_Y) * 100, 2)
xgb_score = round(cross_val(xgbm,train_X_new, train_Y) * 100, 2)


KeyboardInterrupt: 

In [22]:
RF_score

69.36

* **Tabulating the CV Scores for Different Models** 

In [28]:
results = pd.DataFrame({'Model': ['XGB',  'Logistic Regression', 'Random_Forest','KNN','DT'
              ],'Score': [xgb_score , logreg_score,RF_score,knn_score,DT_score]})
results = results.sort_values(['Score'], ascending=[False])
results.head()

NameError: name 'xgb_score' is not defined

# 6. Hyperparameter Tuning for Logistic Regression

Results from the previous step shows that XGBoost and Logistic Regression have the best CV scores (this was also found to be true when the iteration was repeated with 50,100 and 150 features). To reduce the possibility of overfitting, the GridSearchCV method from scikit is used to perform a hyper-paramter optimization on these two models to check if it improves the accuracy of prediction. Below is the code used for a hyperparamter search for logistic regression.

* **Grid Search to Optimize the Parameters** 

In [27]:
grid={"C":np.logspace(-2,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg1=LogisticRegression()
logreg_hp=GridSearchCV(logreg1,grid,cv=3,verbose=0)
logreg_hp.fit(train_X_new,train_Y)

print("tuned hpyerparameters :(best parameters) ",logreg_hp.best_params_)
print("accuracy :",logreg_hp.best_score_)


/Users/prguser/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/prguser/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/prguser/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/prguser/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/prguser/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default so

KeyboardInterrupt: 

* **Modeling with the Best Parameters from GridSearch**

In [31]:
logreg2=LogisticRegression(C=1000,penalty="l2")#Grid Search Result  C:1000 l2 Accuracy :0.902095
logreg2.fit(train_X_new,train_Y)
logreg2_score = round(cross_val(logreg2,train_X_new, train_Y) * 100, 2)
print("Accuracy for logical regression after doing a GridSearchCV: ", logreg2_score)

Accuracy for logical regression after doing a GridSearchCV 75.36


# 7. Hyperparameter Tuning for XGBoost

*The code below is commented out as Kaggle timed out before the grid search is complete for these paramters. 

In [ ]:
#xgbm_h = xgb.XGBClassifier(silent=False, scale_pos_weight=1,learning_rate=0.01, colsample_bytree = 0.4,subsample = 0.8,objective='binary:logistic', 
                    n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)


#params = {
#        'min_child_weight': [1, 5, 10],
#        'gamma': [0.5, 1, 1.5, 2, 5],
#        'subsample': [0.6, 0.8, 1.0],
#        'colsample_bytree': [0.6, 0.8, 1.0],
#        'max_depth': [3, 4, 5]
        }

#xgbm_hp= GridSearchCV(estimator =  xgbm, param_grid = params, scoring='roc_auc',n_jobs=1,iid=False, cv=3)
#xgbm_hp.fit(train_X_new,train_Y)
#xgbm_hp.best_params_, xgbm_hp.best_score_


* **Training XGB Model with Best Parameters from GridSearch **

In [ ]:
#xgbm2 = xgb.XGBClassifier(silent=1, 
#                      scale_pos_weight=1,
#                     learning_rate=0.01,  
#                      colsample_bytree = 0.4,
#                      subsample = 0.8,
#                      objective='binary:logistic', 
#                      n_estimators=1000, 
#                      reg_alpha = 0.3,
#                      max_depth=4, 
#                      gamma=10)
#xgbm2.fit(train_X_new,train_Y)
#xgb2_score = round(cross_val(xgbm2,train_X_new, train_Y) * 100, 2)

# 8. Prediction on Test Data using the Best Performing Model

In [ ]:
#Comment out the models you wont use in this iteration
#logreg_pred = logreg.predict_proba(test_X_new)[:,1]
#logreg2_pred = logreg2.predict_proba(test_X_new)[:,1]
xgb_pred =xgbm.predict_proba (test_X_new)[:,1]
#xgb2_pred =xgbm2.predict_proba (test_X_new)[:,1]


# 8. Generate Output File for Submission

In [ ]:
#The lines below shows you how to save your data in the format needed to score it in the competition
output = pd.DataFrame({'ID_code': test.ID_code,
                      'target': xgb_pred})

output.to_csv('submission.csv', index=False)